<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Data_SRC" data-toc-modified-id="Data_SRC-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data_SRC</a></span></li><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load libraries</a></span></li></ul></li><li><span><a href="#Demo-of-process_mpd" data-toc-modified-id="Demo-of-process_mpd-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Demo of process_mpd</a></span><ul class="toc-item"><li><span><a href="#counting-the-number-of-playlist" data-toc-modified-id="counting-the-number-of-playlist-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>counting the number of playlist</a></span></li><li><span><a href="#get-unique-tracks-in-all-playlist" data-toc-modified-id="get-unique-tracks-in-all-playlist-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>get unique tracks in all playlist</a></span></li><li><span><a href="#create-random-subset-of-1%-of-all-playlists" data-toc-modified-id="create-random-subset-of-1%-of-all-playlists-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>create random subset of 1% of all playlists</a></span></li></ul></li><li><span><a href="#Get-audio-features" data-toc-modified-id="Get-audio-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get audio features</a></span><ul class="toc-item"><li><span><a href="#setup" data-toc-modified-id="setup-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#Read-tracks" data-toc-modified-id="Read-tracks-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Read tracks</a></span></li><li><span><a href="#spotify-web-api" data-toc-modified-id="spotify-web-api-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>spotify-web api</a></span></li><li><span><a href="#merge-data" data-toc-modified-id="merge-data-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>merge data</a></span></li></ul></li></ul></div>

# Preliminaries
## Data_SRC

In [2]:
#location of the dataset
#DATA_SRC='/home/beangoben/ml_data/mpd.v1/data'
DATA_SRC='../data/raw/5k_subset'
# add src scripts to utilize
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load libraries

In [3]:
import os
import json
# utility
from tqdm import tqdm_notebook as tqdm
# scientific python
import numpy as np
import pandas as pd
# nice printing
from pprint import pprint
# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Demo of process_mpd

Located in data/data_utils.py, **process_mpd** will take a directory path, a function to operate on a playlist and a object to store results (could be a dataframe or a dict for example). 

Optional is a *max_n* (max number of slices to operate on) and use *rand=False* for random or sorted files.


## counting the number of playlist

In [37]:
from data_utils import process_mpd

def count_playlist(playlist,results):
    results['count']=results['count']+1
    return

In [ ]:
results={}
results['count']=0
process_mpd(DATA_SRC,count_playlist,results,max_n=10,rand=True)
pprint(results)

## get unique tracks in all playlist

In [9]:
from data_utils import process_mpd, normalize_name

def unique_trackids(playlist,results):
    pid = playlist['pid']
    for track in playlist['tracks']:
        results.append({'title':track['track_name'],'artist':track['artist_name'],'uri':track['track_uri'],'pid':pid})
    return

In [10]:
results=[]
process_mpd(DATA_SRC,unique_trackids,results,max_n=5)
print(len(results))
#pd.DataFrame({'track_uri':list(results)}).to_csv('../data/interim/5k_track_uri.csv',index=False,compression='gzip')

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

335094


In [16]:
df = pd.DataFrame(results)
print(df.shape)
df = df.drop_duplicates('uri')
print(df.shape)
df.to_csv('../data/interim/5k_track_uri.csv',index=False,compression='gzip')

(335094, 4)
(107781, 4)


## create random subset of 1% of all playlists

In [37]:
from data_utils import process_mpd
import random

def coinflip(percent=0.001):
    return random.uniform(0, 1) < percent

def random_subset(playlist,results):
    if coinflip(1):
        results.append(playlist)
    return

In [ ]:
results=[]
process_mpd(DATA_SRC,random_subset,results)
print(len(results))
pprint(results[-1])

 86%|████████▋ | 863/1000 [06:15<00:59,  2.30it/s]

In [27]:
a_dict={"info": {
        "generated_on": "2017-12-03 08:41:42.057563", 
        "slice": "random subset of 0.1% of playlist", 
        "version": "v1"
    }, "playlists": results}
with open('../data/intetim/random_01_percent.json','w') as afile:
    afile.write(json.dumps(a_dict, indent=4))

# Get audio features
## setup

In [5]:
import spotify
import pandas as pd

## Read tracks

In [235]:
df = pd.read_csv('../data/interim/5k_track_uri.csv')
print(df.columns)
print(df.shape)
df.head(1)

Index(['artist', 'pid', 'title', 'uri', 'csv_id', 'artist_uri'], dtype='object')
(108870, 6)


,artist,pid,title,uri,csv_id,artist_uri
0,missy elliott,0,lose control feat ciara & fat man scoop,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0,spotify:artist:2wIVse2owClT7go1WT98tk


## spotify-web api

In [240]:
def get_audio_features(chunk,sp):
    tids = chunk['uri'].apply(lambda x: x.replace('spotify:track:','')).tolist()
    afeatures = sp.audio_features(tids)
    return afeatures

def get_popularity(chunk,sp):
    tids = chunk['uri'].apply(lambda x: x.replace('spotify:track:','')).tolist()
    tracks = sp.tracks(tids)
    pop =[{'popularity':t['popularity']} if t is not None else {'popularity':np.nan} for t in tracks['tracks']]
    return pop

def get_artist(chunk,sp):
    tids = chunk['uri'].apply(lambda x: x.replace('spotify:track:','')).tolist()
    tracks = sp.tracks(tids)    
    arts =[{'artist_uri':t['artists'][0]['uri']} if t is not None else {'artist_uri':None}  for t in tracks['tracks']]
    return arts

def get_artist_genres(chunk,sp):
    tids = chunk['artist_uri'].apply(lambda x: x).tolist()
    arts = sp.artists(tids)
    genres =[{'genres':a['genres']} if a is not None else {'genres':None}  for a in arts['artists']]
    return genres

In [241]:
rows = spotify.df_chunk_apply(df, 50, get_artist_genres)
spotify.replace_none(rows)
a_df = pd.DataFrame(rows)
print(a_df.shape)
a_df.head(1)

HBox(children=(IntProgress(value=0, max=2178), HTML(value='')))


(108870, 1)


,genres
0,"[dance pop, hip hop, hip pop, pop, pop rap, r&..."


In [232]:
af_df = pd.read_csv('../data/interim/5k_track_audiofeatures.csv')
print(af_df.shape)
print(af_df.columns)
af_df

(108870, 19)
Index(['acousticness', 'analysis_url', 'danceability', 'duration_ms', 'energy',
       'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'track_href', 'type', 'uri',
       'valence', 'csv_id'],
      dtype='object')


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,csv_id
0,0.031100,https://api.spotify.com/v1/audio-analysis/0UaM...,0.9040,226864.0,0.8130,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4.0,0.0471,-7.105,0.0,0.1210,125.461,4.0,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.8100,0
1,0.024900,https://api.spotify.com/v1/audio-analysis/6I9V...,0.7740,198800.0,0.8380,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5.0,0.2420,-3.914,0.0,0.1140,143.040,4.0,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.9240,1
2,0.002380,https://api.spotify.com/v1/audio-analysis/0WqI...,0.6640,235933.0,0.7580,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2.0,0.0598,-6.583,0.0,0.2100,99.259,4.0,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.7010,2
3,0.202000,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.8910,267267.0,0.7140,1AWQoqb9bSvzTjaLralEkT,0.000234,4.0,0.0521,-6.055,0.0,0.1400,100.972,4.0,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.8180,3
4,0.056100,https://api.spotify.com/v1/audio-analysis/1lzr...,0.8530,227600.0,0.6060,1lzr43nnXAijIGYnCT8M8H,0.000000,0.0,0.3130,-4.596,1.0,0.0713,94.759,4.0,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.6540,4
5,0.021200,https://api.spotify.com/v1/audio-analysis/0XUf...,0.8810,250373.0,0.7880,0XUfyU2QviPAs6bxSpXYG4,0.000000,2.0,0.0377,-4.669,1.0,0.1680,104.997,4.0,https://api.spotify.com/v1/tracks/0XUfyU2QviPA...,audio_features,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0.5920,5
6,0.257000,https://api.spotify.com/v1/audio-analysis/68vg...,0.6620,223440.0,0.5070,68vgtRHr7iZHpzGpon6Jlo,0.000000,5.0,0.0465,-8.238,1.0,0.1180,86.412,4.0,https://api.spotify.com/v1/tracks/68vgtRHr7iZH...,audio_features,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0.6760,6
7,0.158000,https://api.spotify.com/v1/audio-analysis/3BxW...,0.5440,225560.0,0.8230,3BxWKCI06eQ5Od8TY2JBeA,0.000000,2.0,0.2680,-4.318,1.0,0.3200,210.750,4.0,https://api.spotify.com/v1/tracks/3BxWKCI06eQ5...,audio_features,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0.4340,7
8,0.273000,https://api.spotify.com/v1/audio-analysis/7H6e...,0.7130,271333.0,0.6780,7H6ev70Weq6DdpZyyTmUXk,0.000000,5.0,0.1490,-3.525,0.0,0.1020,138.009,4.0,https://api.spotify.com/v1/tracks/7H6ev70Weq6D...,audio_features,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0.7340,8
9,0.103000,https://api.spotify.com/v1/audio-analysis/2Ppr...,0.7280,235213.0,0.9740,2PpruBYCo4H7WOBJ7Q2EwM,0.000532,4.0,0.1750,-2.261,0.0,0.0665,79.526,4.0,https://api.spotify.com/v1/tracks/2PpruBYCo4H7...,audio_features,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0.9650,9


In [132]:
display(af_df.loc[a_df[pd.isnull(a_df['artist_uri'])].index]['track_href'])
display(df.loc[a_df[pd.isnull(a_df['artist_uri'])].index])

20622     https://api.spotify.com/v1/tracks/656TZlNdVe90...
28830     https://api.spotify.com/v1/tracks/5GiU7GOYjDH2...
36627     https://api.spotify.com/v1/tracks/2ZF3gpgGPjOf...
45507     https://api.spotify.com/v1/tracks/2odGdanrUsgu...
64596     https://api.spotify.com/v1/tracks/6tf7ENqpF89X...
78710     https://api.spotify.com/v1/tracks/0fJ6UuZ7onqc...
79513     https://api.spotify.com/v1/tracks/0Jb7g80rInlj...
95918     https://api.spotify.com/v1/tracks/1HubPEsZLy8R...
102653    https://api.spotify.com/v1/tracks/6CNzVurjdLR4...
Name: track_href, dtype: object

,artist,pid,title,uri,csv_id,artist_uri
20622,son palenque,497,palenque palenque,spotify:track:656TZlNdVe90zHvmebFt9U,20622,NaN
28830,coldplay,782,adventure of a lifetime,spotify:track:5GiU7GOYjDH2yp7fMf9w9j,28830,NaN
36627,bing crosby,1065,white christmas,spotify:track:2ZF3gpgGPjOfi1yTWGX6Bm,36627,NaN
45507,dionne warwick,1425,walk on by - live,spotify:track:2odGdanrUsgu0d2i2Q5XoT,45507,NaN
64596,thomas newson,2331,wildfire - radio edit,spotify:track:6tf7ENqpF89Xox3xyNPGMv,64596,NaN
78710,alex aiono,3114,i hate you i love,spotify:track:0fJ6UuZ7onqcldGr8HT5QV,78710,NaN
79513,marvin gaye,3158,third world girl,spotify:track:0Jb7g80rInlj3fa6W65Bp2,79513,NaN
95918,mat joe,4140,make a living,spotify:track:1HubPEsZLy8R1HuCzMNBCd,95918,NaN
102653,jake spooner,4567,rivals,spotify:track:6CNzVurjdLR4seIfAsRxai,102653,NaN


In [219]:


df = fix_artist_uri(df,pd.isnull(df['artist_uri']),sp)

Fixed 0 out of 0


In [112]:
#df.merge(a_df[['artist_uri']],left_index=True,right_index=True).to_csv('../data/interim/5k_track_uri.csv',index=False)